In [17]:
# imports
import jax
import numpy as np

from defaults import default_seed

In [18]:
class QuadraticProblem:
    """
    A class describing an unconstrained quadratic problem:
    0.5 x^TAx^T + b^Tx, where the matrix A is non-negative defined, x \in R^n
    """
    n: int = 1           # problem dimensionality
    A = None             # A-matrix: np.array of shape (n,n)
    b = None             # b-vector: np.array of shape (n,) 
    def __init__(self, n: int = 2, A = None, b = None):
        np.random.seed(default_seed)
        self.n = n
        if A is None:
            self.A = self.__get_random_matrix(self.n)
        else:
            self.A = A
        if b is None:
            self.b = self.__get_random_vector(self.n)
        else:
            self.b = b
        
    
    def f(self, x):
        """
        Target function
        x: np.array of shape (n, )
        return: float
        """
        return 0.5 * x.T @ self.A @ self.x + self.b.T @ x
    
    
    def __get_random_matrix(self, n: int = 2):
        """
        Returns a non-negative defined matrix of size (n, n)
        """
        B = np.random.rand(n, n)
        return B.T @ B
    
    def __get_random_vector(self, n: int = 2):
        """
        Returns a random vector: np.array of shape (n,)
        """
        return np.random.rand(n)

In [21]:
n = 2
qp = QuadraticProblem(n=n)
print(qp.A, qp.b)

[[0.67609655 0.62907433]
 [0.62907433 0.58994988]] [0.37223648 0.0304395 ]


In [ ]:
class Benchmark:
    """
    A class that provides the benchmarking of different optimization methods on a quadratic problem (convex).
    """
    def __init__(self):
        pass
    